# XGBOOST Training
The full data set is expected to exceed the memory of a single GPU so a distributed XGBOOST training strategy is applied. In situations where data fits nicely on a single machine, avoiding distributed training is recommended for improved performance. Prototyping the ML model involves running a few years of data through “3-etl_test_train_split_tgt_enc.ipynb” to generate the fully encoded wide-form tables for the various split datasets. 

We would like to predict DELAY_CAUSES_ENC, which contains multiple delay causes (LATE_AIRCRAFT_DELAY, CARRIER_DELAY, NAS_DELAY, WEATHER_DELAY, SECURITY_DELAY). SECURITY_DELAY and WEATHER_DELAY occur infrequently so they are combined into OTHER_DELAY. The first letter of each delay cause is used in a binary sequence (e.g., LCNO) in the DELAY_CAUSES_ENC column. The overall result is 4 unique delay causes with 16 possible combinations. To further reduce the number of meta-classes, a 95th percentile cutoff filter is applied to the ranked frequency of each meta-class. Infrequently occurring classes all end with “O” (for OTHER) in their DELAY_CAUSES_ENC and are encoded with the number 8 in the final DELAY_CAUSES_ENC representation. Including the no-delay case (“----”, or 0), there are a total of 9 meta-classes to consider.  

XGBOOST’s default treatment of multi-class classification problems involve decomposing the problem using One-vs-Rest (OVR). This decomposition strategy treats the multi-class classification problem as N-binary classification sub-problems, with N being the number of distinct categories to be predicted (N=9 in our case). Reducing the number of classes while still being able to faithfully represent the data is desirable. Because N different binary classification models are created, inferencing and model explainability becomes much more computationally intensive. New data needs to pass through each binary classification sub-model and then blended into a final prediction.    

Challenges: (1) severe class imbalance; (2) large amounts of class overlap between certain classes. Class imbalance can be addressed via applying class weights during xgboost training or implementing resampling techniques. Combining related classes may help with the class overlap problem.  

In [1]:
import gc
import os
import sys
import glob
import shutil
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pandas.api.types import CategoricalDtype
import numpy as np
from time import time
from datetime import datetime

import dask
import dask.dataframe as dd
from dask.distributed import Client, wait, progress, get_worker

import sklearn
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost.dask import DaskDMatrix

print('xgboost version', xgb.__version__)

run_type = 'cpu'
max_depth = 15
num_boost_round = 200
score_metric = 'auc'
storage_backend = 'local'

test_size = 1/17 # Non-zero to pass data split check. 
   
study_arpt = 'NAS'

if study_arpt == 'NAS':
    # NAS processing excludes weather. Has additional cols for arrival/departure airports.
    pred_model = 'multi_class'
    label_col = 'DELAY_CAUSES_ENC'
    excluded_features = [label_col, 'cv_idx', 'class_weight', 'UID', 'ARR_DEL15', 'DEP_DEL15', 'YYYYMM']
else:
    pred_model = 'binary_class'
    label_col = 'ARR_DEL15'
    excluded_features = [label_col, 'cv_idx', 'UID', 'DEL_ARR_PER_QTHR', 'DEL_DEP_PER_QTHR', 'DEP_DEL15', 'ARR_DEL']

if pred_model == 'binary_class':
    xgb_objective = 'binary:logistic'
elif pred_model == 'multi_class':
    # xgboost auc docs mentioned that: "When used with multi-class classification, objective should be multi:softprob instead of multi:softmax, 
    # as the latter doesn’t output probability. Also the AUC is calculated by 1-vs-rest with reference class weighted by class prevalence."
    xgb_objective = 'multi:softprob'
# elif pred_model == 'regression':
#     label_col = 'ARR_DELAY' # Regression model
#     xgb_objective = 'reg:squarederror'
    
enc_output_dir = './data/encoded/split/'+study_arpt

xgb_model_name = 'xgb_'+run_type+'_airline_delay_'+study_arpt+'_max_depth_'+str(max_depth)


if run_type == 'gpu':
    from dask_cuda import LocalCUDACluster
    
    # Run on all available GPU on same machine:
    cluster = LocalCUDACluster(threads_per_worker=16, memory_limit='128GB')
    client = Client(cluster)
    
    # Run single GPU:
#     client = Client(n_workers=1, threads_per_worker=32)
elif run_type == 'cpu':
    client = Client(n_workers=1, threads_per_worker=32)
#     client = Client(n_workers=2, threads_per_worker=16)   
    
client

xgboost version 1.5.0-dev


Connection method: Cluster object,Cluster type: LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Status: running,Using processes: True
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 32,Total memory: 251.65 GiB
Comm: tcp://127.0.0.1:36873,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 251.65 GiB
Comm: tcp://127.0.0.1:46331,Total threads: 32
Dashboard: http://127.0.0.1:42325/status,Memory: 251.65 GiB
Nanny: tcp://127.0.0.1:34799,


In [2]:
def dask_xgb_train_from_enc(client, run_type, data_path, label_col, xgb_params, xgb_model_name):
    """
    Use dask + xgboost for training against already encoded data.
    """
    if run_type == 'cpu':
        ddf_enc = dd.read_parquet(data_path)
    elif run_type == 'gpu':
        import dask_cudf
        import cupy as cp
        ddf_enc = dask_cudf.read_parquet(data_path)
    else:
        raise ValueError('Select run_type of cpu or gpu.')

    tic = time()
    ddf_enc = client.persist(ddf_enc)
#     wait([ddf_enc])

    feature_cols = [cc for cc in ddf_enc.columns if cc not in excluded_features]
    print('Feature columns:', feature_cols)
    print('  Number of feature columns:', len(feature_cols))
    
    X = ddf_enc[feature_cols].astype('float32')
    y = ddf_enc[label_col].astype('float32')
    
    # Per-instance weight required due to severe class imbalance in One-vs-Rest decomposition.
    # Weights precalculated during ETL on training set alone to avoid leakage.
    weight = ddf_enc['class_weight'].astype('float32')
    
#     # Persist data:
#     X = client.persist(X)
#     y = client.persist(y)
#     weight = client.persist(weight)
#     wait([X, y, weight])

    # Use XGBOOST API:
    if run_type == 'gpu':
        # See example: https://github.com/dmlc/xgboost/blob/master/demo/dask/gpu_training.py
        # `DaskDeviceQuantileDMatrix` is used instead of `DaskDMatrix`, be careful
        # that it can not be used for anything else than training.
        dtrain = xgb.dask.DaskDeviceQuantileDMatrix(client, X, y, weight=weight)
    else:
        dtrain = DaskDMatrix(client, X, y, weight=weight)
    
#     del ddf_enc, X, y # Cleanup
    data_load_time = np.round(time() - tic, 2)

    tic = time()
    xgb_model = xgb.dask.train(client, xgb_params, dtrain, num_boost_round=num_boost_round)['booster']
    train_model_time = np.round(time() - tic, 2)

    # Save xgb model to disk:
    tic = time()
    xgb_model.save_model(xgb_model_name+'.model') # Save model to current dir
    xgb_model.save_model('/tmp/'+xgb_model_name+'.json') # Experimental JSON format. To be nested in output file with timings/metrics.

    # Move local xgb-model to hdfs if xgb_hdfs_output_dir specified:
    if (storage_backend == 'hdfs'):
        if xgb_hdfs_output_dir != None:
            with open(xgb_model_name+'.model','rb') as f:
                hdfs.upload(os.path.join(xgb_hdfs_output_dir, xgb_model_name+'.model'), f)
    export_model_time = np.round(time() - tic, 2)

    metrics = {
        'data_load_time': data_load_time,
        'train_model_time': train_model_time,
        'export_model_time': export_model_time
        }

    del dtrain
    gc.collect() # Manual garbage collection needed since python may not clean up GPU resources automatically.
    return(metrics)


# xgboost params:
xgb_params = { 
    'random_state': 0,
    'eta': 0.1,
    'gamma': 0.1,
    'max_depth': max_depth, 
#     'num_boost_round': num_boost_round, # Number of boosting rounds: num_boost_round in xgboost, numRound in xgboost4j, n_estimators in scikit-learn.
    'max_leaves': 4*256, # Maximum number of nodes to be added. Only relevant when grow_policy=lossguide is set.
    'objective': xgb_objective,
    'eval_metric': 'auc', # Applied to eval_set/test_data if provided
    'booster': 'gbtree',
}

if run_type == 'cpu':
    tree_method = 'hist'
elif run_type == 'gpu':   
    tree_method = 'gpu_hist'

xgb_params['tree_method'] = tree_method

if study_arpt == 'NAS':
    y_tmp = dd.read_parquet(enc_output_dir+'/train', columns=label_col)
    
    # Assume that classes have been sequentially encoded:
    num_class = y_tmp.max().compute() + 1
    print('Number of classes detected: ', num_class)
    xgb_params['num_class'] = num_class

print('Starting xgboost '+run_type+' '+pred_model+' training....')

tic = time()

# Large amount of memory required to train on full 16 yrs of data. Reduce number of years used during training on single node tests.
training_metrics = dask_xgb_train_from_enc(client, run_type, enc_output_dir+'/train', label_col, xgb_params, xgb_model_name)

t_dask_train = np.round(time() - tic, 2)

training_metrics['total_time'] = t_dask_train
print(training_metrics)
print('Dask xgboost '+run_type+' training time:', '{:0.2f}'.format(t_dask_train) + 's')

# Running multi-class is similar in timing to running N_class-times binary classification due to One-vs-Rest problem decomposition.

Number of classes detected:  9
Starting xgboost cpu multi_class training....
Feature columns: ['YEAR', 'DEP_DELAY', 'ARR_DELAY', 'TAXI_OUT', 'TAXI_IN', 'CRS_ELAPSED_TIME', 'ACTUAL_ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DISTANCE_GROUP', 'DEP_ARPT_LAT', 'DEP_ARPT_LON', 'DEP_ARPT_ELV_FT', 'ARR_ARPT_LAT', 'ARR_ARPT_LON', 'ARR_ARPT_ELV_FT', 'IS_HOLIDAY', 'IS_WEEKEND', 'DEP_ARPT_TYPE_large_airport', 'DEP_ARPT_TYPE_medium_airport', 'DEP_ARPT_TYPE_small_airport', 'ARR_ARPT_TYPE_large_airport', 'ARR_ARPT_TYPE_medium_airport', 'ARR_ARPT_TYPE_small_airport', 'QUARTER_cos', 'QUARTER_sin', 'MONTH_cos', 'MONTH_sin', 'DAY_OF_MONTH_cos', 'DAY_OF_MONTH_sin', 'DAY_OF_YEAR_cos', 'DAY_OF_YEAR_sin', 'DAY_OF_WEEK_cos', 'DAY_OF_WEEK_sin', 'CRS_DEP_TIME_HR_cos', 'CRS_DEP_TIME_HR_sin', 'CRS_DEP_TIME_QTHR_cos', 'CRS_DEP_TIME_QTHR_sin', 'CRS_ARR_TIME_HR_cos', 'CRS_ARR_TIME_HR_sin', 'CRS_ARR_TIME_QTHR_cos', 'CRS_ARR_TIME_QTHR_sin', 'ORIGIN_ARR_PER_QTHR', 'ORIGIN_CRS_ARR_PER_QTHR', 'ORIGIN_DEL_ARR_PER_QTHR', 'ORI

[23:48:51] task [xgboost.dask]:tcp://127.0.0.1:46331 got new rank 0


{'data_load_time': 21.35, 'train_model_time': 3250.6, 'export_model_time': 3.76, 'total_time': 3276.09}
Dask xgboost cpu training time: 3276.09s


In [3]:
def score_pred(run_type, y_true, y_pred, score_metric):
    """
    Prediction scoring function.
    """
    if pred_model == 'multi_class':
        # Scoring for multi-class is done on CPU only. cuML roc_auc_score doesn't not work with multi-class yet.
        # Work-around is to use CPU for scoring via converting gpu-dataframe to pandas.
        if run_type == 'gpu':
            import cupy
            y_true = cupy.asnumpy(y_true)
            y_pred = cupy.asnumpy(y_pred)
            
            # Update run_type to bypass GPU methods.
            run_type = 'cpu'
    
    if run_type == 'gpu':
#         import dask.dataframe as hw # Has internal switching for compatibility. Takes a long time to perform AUC score, but fast at prediction.
        from cuml.metrics import roc_auc_score
        from cuml.metrics.accuracy import accuracy_score
        # TODO: inplace scoring to pair with inplace prediction? Only works with certain methods.
    else:
        # TODO: need parallelized roc_auc_score computation for auc.
        from sklearn.metrics import roc_auc_score # Single-threaded?
#         from sklearn.metrics import accuracy_score # Single-threaded
        from dask_ml.metrics import accuracy_score # CPU parallelized

    if score_metric == 'auc':
        if pred_model == 'multi_class':
            # xgb multi-class classification uses One-vs-Rest (or One-vs-All): 
            score_value = roc_auc_score(y_true, y_pred, multi_class='ovr', average='macro')
            
            # Only support average param of: 
                # 'macro' (default): Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.)
                # 'weighted': Calculate metrics for each label, and find their average, weighted by support (the number of true instances for each label).
                
            # Use average='macro' so model takes extreme imbalance of minority classes into consideration within score.
        else:
            # AUC uses probabilities. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html
            score_value = roc_auc_score(y_true, y_pred)
    elif score_metric == 'acc':
        # Accuracy score uses threshold value due to ==. https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
        score_value = accuracy_score(y_true.round(), y_pred.round()) # y_pred needs to be converted explicitly. Dask doesn't know shape of y, y_pred if dask.dataframe.
    return(score_value)

def dask_xgb_infer(client, run_type, model_file, data_path, label_col, score_metric='auc'):
    """
    Use dask + xgboost for inferencing against already encoded data.
    """    
    if run_type == 'gpu':
        import dask_cudf as hw # Needed for xgb.dask.inplace_predict(). 
#         import dask.dataframe as hw # Has internal switching for compatibility. Takes a long time to perform AUC score, but fast at prediction.
    else:
        import dask.dataframe as hw
    
    model = xgb.Booster(model_file=model_file+'.model')

    tic = time()
    ddf_enc = hw.read_parquet(data_path).persist()

    # Format data for xgboost:
    feature_cols = [cc for cc in ddf_enc.columns if cc not in excluded_features]
#     print(feature_cols)
    X = ddf_enc[feature_cols].astype('float32')
    y = ddf_enc[label_col].astype('float32')

#     # Persist data:
#     X = client.persist(X)
#     y = client.persist(y)
#     wait([X, y])
    data_load_time = np.round(time() - tic, 2)

    # Run predictions:
    tic = time()
    model.set_param('predictor', run_type + '_predictor')
    model = client.scatter(model, broadcast=True)

    # See https://xgboost.readthedocs.io/en/latest/tutorials/dask.html#running-prediction:
#     y_pred = xgb.dask.predict(client, model, xgb.dask.DaskDMatrix(client, X, y)) # More efficient to avoid xgb.dask.DaskDMatrix()
#     y_pred = xgb.dask.predict(client, model, X) # Use inplace_predict() can sometimes be faster.

    # IMPORTANT: X.values required to get consistent CPU scoring. Issue with column ordering in dask.dataframe?
    y_pred = xgb.dask.inplace_predict(client, model, X.values) # Use inplace_predict() can sometimes be faster.

    # Force computation of y's required for libraries without dask.dataframe support:
    y_pred = y_pred.compute()
    y = y.compute()
    wait([y_pred, y])

    predict_time = np.round(time() - tic, 2)

    # Score predictions:
    print('Scoring predictions in '+ data_path)
    tic = time()
    score_value = score_pred(run_type, y, y_pred, score_metric)
    score_time = np.round(time() - tic, 2)

    # TODO: have inference run distributively. Need roc_auc_score equivalent for dask-gpu. Maybe dask_cudf has it?
    metrics = {
        'data_load_time': data_load_time,
        'predict_time': predict_time,
        'score':
            {score_metric: score_value,
            score_metric+'_time': score_time
                }
        }

    del ddf_enc, X, y, y_pred, model
    gc.collect()
    return(metrics)


# Bypass original xgb_model_name to run inference of same model along CPU/GPU code path:
# run_type = 'cpu'
# xgb_model_name = 'xgb_'+run_type+'_airline_delay_'+study_arpt
# xgb_model_name = 'xgb_cpu_airline_delay_'+study_arpt


# TODO: Update auc scoring for multi-class problem. cuML roc_auc_score doesn't handle multi-class currently.
# https://github.com/rapidsai/cuml/blob/46174b7/python/cuml/metrics/_ranking.py#L119. Should use sklearn directly. Or switch over to xgboost scoring?


print('Starting '+run_type+' '+pred_model+' inference rounds....')
tic = time()
if test_size == 0:
    inference_metrics_train = dask_xgb_infer(client, run_type, xgb_model_name, enc_output_dir+'/train', label_col, score_metric=score_metric)
    inference_metrics_test = []
else:
    # Inference against entire data set in two steps:
    inference_metrics_train = dask_xgb_infer(client, run_type, xgb_model_name, enc_output_dir+'/train', label_col, score_metric=score_metric)
    inference_metrics_test = dask_xgb_infer(client, run_type, xgb_model_name, enc_output_dir+'/test', label_col, score_metric=score_metric)
t_dask_infer = np.round(time() - tic, 2)

end_time_str = datetime.now().strftime("%m/%d/%Y %H:%M:%S")

print('Inference against training data: ', inference_metrics_train)
print('Inference against (hold-out) test data: ', inference_metrics_test)
print('Total prediction time:', '{:0.2f}'.format(t_dask_infer) + 's')

Starting cpu multi_class inference rounds....
Scoring predictions in ./data/encoded/split/NAS/train
Scoring predictions in ./data/encoded/split/NAS/test
Inference against training data:  {'data_load_time': 0.07, 'predict_time': 163.51, 'score': {'auc': 0.9899401180320224, 'auc_time': 147.97}}
Inference against (hold-out) test data:  {'data_load_time': 0.04, 'predict_time': 26.6, 'score': {'auc': 0.982263427248808, 'auc_time': 14.12}}
Total prediction time: 352.55s
